In [2]:
import xarray as xr 
import dask as da
import os
import matplotlib.pyplot as plt
import numpy as np

## Constantes

In [1]:
t_s = -1.0
t_r = 3.0

A_max = 20.0
t_a = -10.0
A_0 = 0.2
a = 0.2
b = 0.1

In [3]:
#ruta_actual = os.getcwd()

# Construye la ruta completa a la carpeta "datos"
#ruta_datos = os.path.join(ruta_actual, "Tesina", "Data")

# Ahora puedes acceder a los archivos .nc en la carpeta "datos"
#archivo_pr = os.path.join(ruta_datos, "CR2MET_pr_v2.0_day_1979_2020_005deg.nc")
#archivo_t2m = os.path.join(ruta_datos, "CR2MET_t2m_v2.0_day_1979_2020_005deg.nc")

#print(os.path.exists(archivo_pr))
#print(os.path.exists(archivo_t2m))

ds_pr = xr.open_dataset(r'C:/Users/Josemanuel/Desktop/Tesina/Data/CR2MET_pr_v2.0_day_1979_2020_005deg.nc')
ds_t2m = xr.open_dataset(r'C:/Users/Josemanuel/Desktop/Tesina/Data/CR2MET_t2m_v2.0_day_1979_2020_005deg.nc')

In [21]:
def parti(i,j,T=False, P=False):
    if T:
        
        T = ds_t2m['t2m'][i:j,:,:]

        # Elimina los valores NaN
        #t_sn_part = xr.where(np.isnan(T), 0, T)
        
        return T.values
    
    if P:
        
        p = ds_pr['pr'][i:j,:,:]

        # Elimina los valores NaN
        #p_sn = xr.where(np.isnan(p), 0, p)

        return p.values

In [6]:
def modelo_completo_final(temp_sn, pres_sn, t_s, t_r, t_a, A_0, a, b, A_max):

    #temp_sn = temp_sn.chunk(500000)
    #pres_sn = pres_sn.chunk(500000)

    # KSF 

    mask_nan = np.isnan(temp_sn)

    # Inicializar k con NaN en todas partes
    #k = np.zeros_like(temp_sn, dtype='float16')
    k = np.full_like(temp_sn, np.nan, dtype='float16')

    # Aplicar las condiciones a los valores no NaN
    k[~mask_nan] = 0.0  # Inicializar con 0.0 en lugares donde temp_sn no es NaN
    k[temp_sn <= t_s] = 1.0
    mask = (t_s <= temp_sn) & (temp_sn <= t_r)
    k[mask] = (t_r - temp_sn[mask]) / (t_r - t_s)
    k[temp_sn > t_r] = 0.0

    # A_*

    #A_ast = np.zeros_like(temp_sn, dtype='float16')
    A_ast = np.full_like(temp_sn, np.nan, dtype='float16')
    A_ast[~mask_nan] = 0.0
    A_ast[temp_sn <= t_a] = 0.0
    mask = (t_a <= temp_sn) & (temp_sn <= 0)
    A_ast[mask] = A_0*(t_a - temp_sn[mask]) / (t_a)
    mask2 = (temp_sn > 0)
    A_ast[mask2] =   a*(temp_sn[mask2]**2) + (b*temp_sn[mask2]) + A_0


    # SNOWFALL

    S_ = k * pres_sn 

    # ABLATION 

    A_ = np.full_like(S_, np.nan, dtype='float16')
    #A_ = np.zeros(S_.shape, dtype='float16')
    
    # DELTA SWE

    #SWE = np.zeros(S_.shape, dtype='float16')  # Initialize SWE with zeros of the same shape as S_
    SWE = np.full_like(S_, np.nan, dtype='float16')

    for i in range(S_.shape[0]):
        if i == 0:
            A_[i, ~mask_nan[i, :, :]] = np.minimum(np.minimum(A_max, A_ast[i, ~mask_nan[i, :, :]]), S_[i, ~mask_nan[i, :, :]])
            SWE[i, ~mask_nan[i, :, :]] = S_[i, ~mask_nan[i, :, :]] - A_[i, ~mask_nan[i, :, :]]
        else:
            A_[i, ~mask_nan[i, :, :]] = np.minimum(np.minimum(A_max, A_ast[i, ~mask_nan[i, :, :]]), S_[i, ~mask_nan[i, :, :]])
            SWE[i, ~mask_nan[i, :, :]] = SWE[i-1, ~mask_nan[i, :, :]] + (S_[i, ~mask_nan[i, :, :]] - A_[i, ~mask_nan[i, :, :]])

    return SWE, S_, A_

In [7]:
# T = ds_t2m['t2m'][:,:,:]
# p = ds_pr['pr'][:,:,:]

t_part = parti(0,15737, T=True)
p_part = parti(0,15737, P=True)

In [9]:
swe_c, snow_f, abla_c = modelo_completo_final(t_part, p_part, t_s, t_r, t_a, A_0, a, b, A_max) #swe_c, snow_f_c, abla_c, k

In [11]:
swe_c = swe_c.astype('float32')
snow_f = snow_f.astype('float32')
abla_c = abla_c.astype('float32')
#A_ast = A_ast.astype('float32')
#k = k.astype('float32')

ds_swe = xr.Dataset(
    {
        'swe': (['time', 'lat', 'lon'], swe_c, {'units': 'mm/day'}),
        'snow_f': (['time', 'lat', 'lon'], snow_f, {'units': 'mm/day'}),
        'abla': (['time', 'lat', 'lon'], abla_c, {'units': 'mm/day'}),
         #'A_ast': (['time', 'lat', 'lon'], A_ast, {'units': 'mm/day'}),
         #'k': (['time', 'lat', 'lon'], k, {'units': 'mm/day'})
    },
    coords={
        'time': ds_pr['time'],
        'lat': ds_pr['lat'],
        'lon': ds_pr['lon']
    }
)

# Agrega atributos adicionales si es necesario
#ds_swe.attrs['units'] = 'mm'  # Ajusta las unidades según sea necesario

# Guarda el conjunto de datos en un archivo NetCDF
ds_swe.to_netcdf(r'C:/Users/Josemanuel/Desktop/Tesina/Resultados/SWE_day_1979_2020_7.nc')